## Analysis of Tf633 Intensity in Neural Crest Explants - Ceramide Rescue

In [1]:
# Import necessary packages
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats
import scikit_posthocs

# Import plotting packages
import iqplot
import bokeh.io
from bokeh.io import output_file, show
from bokeh.layouts import column, row
bokeh.io.output_notebook()

Loading BokehJS ...

### Initial import, concatenation, and annotating of data from Fiji

In [64]:
# Define path to directory with measurements
path = os.path.abspath('raw_source_data/')
df_summary = pd.DataFrame()
list_summary = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*_Intensity.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name, then parse experiment details from Image name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    (df['Date'], df['Treatment'], df['Ceramide'], df['Explant'], df['Stains'], 
             df['FOV'], df['Measurement']) = zip(*df['Image'].map(lambda x:x.split('_')))
    (df['Channel'], df['ROI']) = zip(*df['Label'].map(lambda x:x.split(':')))

    # Drop unnecessary columns for tidiness
    df = df.drop(['Label', 'RawIntDen'], axis = 1)
    
    # Compile data
    list_summary.append(df)
df_summary = pd.concat(list_summary, sort=False)

# Preview dataframe to confirm import successful
df_summary.head()

### Process intensities to generate CTCF
df_tf633 = df_summary.loc[df_summary['Channel'] == 'Tf633']

# Get a list of target genes measured
image_list = df_summary.Image.unique().tolist()


# Initialize for final dataframe collection
full_results = pd.DataFrame()
full_results_list = []

# Loop through image dataset:
for image in image_list:
    df_image = df_tf633.loc[df_tf633['Image'] == image][['Date','Treatment','Ceramide','Explant',
                                                             'FOV','ROI','Area','Mean','IntDen']]
    background_mean = df_image.loc[df_image['ROI'] == 'background']['Mean']
    df_image_processed = df_image.loc[df_image['ROI'] != 'background'].copy()
    df_image_processed['CTCF'] = (df_image_processed['IntDen'] 
                                  - (df_image_processed['Area'] * float(background_mean)))
    full_results_list.append(df_image_processed)
    
# Assemble and view the final results
full_results = pd.concat(full_results_list,sort=False).reset_index().drop('index', axis=1)
full_results['Condition'] = full_results['Treatment']+'_'+full_results['Ceramide']
full_results.head()

,Date,Treatment,Ceramide,Explant,FOV,ROI,Area,Mean,IntDen,CTCF,Condition
0,20221022,SMPD3MO,C12Cer;10uM,Explant3,FOV5,0003-0319-0367,739.419,219.504,162305.167,75641.563105,SMPD3MO_C12Cer;10uM
1,20221022,SMPD3MO,C12Cer;10uM,Explant3,FOV5,0001-0403-0472,714.358,556.204,397328.532,313602.202610,SMPD3MO_C12Cer;10uM
2,20221022,SMPD3MO,C12Cer;10uM,Explant3,FOV5,0001-0364-0856,536.020,1035.307,554945.362,492121.137900,SMPD3MO_C12Cer;10uM
3,20221022,SMPD3MO,C12Cer;10uM,Explant3,FOV5,0001-0668-0522,428.149,276.620,118434.684,68253.480455,SMPD3MO_C12Cer;10uM
4,20221022,SMPD3MO,C12Cer;10uM,Explant3,FOV5,0001-0758-0603,325.575,590.948,192398.068,154239.050125,SMPD3MO_C12Cer;10uM


### Plot results

In [65]:
# Figure out conditions list
full_results['Condition'].unique().tolist()

['SMPD3MO_C12Cer;10uM',
 'SMPD3MO_C12Cer;50uM',
 'SMPD3MO_DMSO',
 'ControlMO_DMSO']

In [70]:
plotting_metric = 'CTCF'
categories = 'Condition'

treatment_list = [
     'ControlMO_DMSO',
     'SMPD3MO_DMSO',
     'SMPD3MO_C12Cer;10uM',
     'SMPD3MO_C12Cer;50uM',
]

df_subset = full_results
df_subset = df_subset.loc[df_subset['Condition'].isin(treatment_list)]

test_stripbox = iqplot.stripbox(
       data = df_subset, q=plotting_metric, q_axis='y'
        ,cats=categories
        ,order=treatment_list
        ,spread='jitter' ,jitter_kwargs=dict(width=0.25) 
        ,box_kwargs=dict(line_color='black', line_width=1.5)
        ,whisker_kwargs=dict(line_color='black', line_width=1.5)
        ,median_kwargs=dict(line_color='maroon', line_width=4)
        ,top_level='box'
        ,y_axis_type='log'
)

test_stripbox.xaxis.major_label_orientation = 45
show(row(test_stripbox))

In [71]:
# Pivot and export plotting data to bring to Prism
df_pivot = df_subset.copy()
df_pivot = df_pivot.pivot(index=['Date', 'Treatment', 'Ceramide', 'Explant', 'FOV', 'ROI'], 
                          columns='Condition', values='CTCF')
df_pivot = df_pivot[['ControlMO_DMSO', 'SMPD3MO_DMSO', 'SMPD3MO_C12Cer;10uM', 'SMPD3MO_C12Cer;50uM']]
df_pivot.to_csv('source_data.csv')